In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import KFold
import lightgbm as lgbm# .loc[df_test.sku_name == new_id_test[0]]
from sklearn.preprocessing import LabelEncoder as LE
import seaborn as sns
import scipy.stats as ss
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
import seaborn as sns
from matplotlib import pyplot as plt
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["Feature", "Importance"]].groupby("Feature").mean().sort_values(by="Importance", ascending=False)[:10].index
    best_features = feature_importance_df_[["Feature", "Importance"]].groupby("Feature").mean().sort_values(by="Importance", ascending=False)[:50]
    best_features.reset_index(inplace=True)
    print(best_features.dtypes)
    plt.figure(figsize=(8, 10))
    sns.barplot(x="Importance", y="Feature", data=best_features)
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
ss = pd.read_csv('SampleSubmission.csv')
dict_s = pd.read_csv('DataDictionary.csv')

In [307]:
# test
import calendar
def count_weekends(m,y):
    day_to_count = calendar.SUNDAY

    matrix = calendar.monthcalendar(y,m)

    num_days = sum(1 for x in matrix if x[day_to_count] != 0)
    return num_days%4

In [308]:
# test['Weeks'] = test.apply(lambda x: count_weekends(x.month, x.year), axis=1)
# train

#Tests

In [309]:
# test
# exp = (train.groupby(['sku_name','year','month'])['price'].mean()//10).astype('int16').reset_index()
# exp = exp.groupby(['sku_name'])['price'].nunique().reset_index()

In [310]:
# exp[exp.price == 3].sku_name.isin(test.sku_name)
# i = 12
# train.groupby('sku_name').get_group(test.sku_name.unique()[i]).sort_values(by = ['year','month'])[['price','cum_disc']]

In [311]:
# train.columns
# test['cum_disc'] =1

In [312]:
# train.groupby(['sku_name','year','month'])['disc_month'].sum().idxmax()


In [313]:
# exp.groupby('sku_name').(.sort_values(by = ['year','month'])['price'].diff())

In [314]:
# train.columns

In [315]:
# train.groupby('sku_name').get_group('ALONRTIEARLE').sort_values(by = ['year','month'])[['year','month','price','cum_disc']]

In [316]:
# test.shape[0]/train.shape[0]

In [317]:

target = 'sellin'
prediction_columns = test.columns.tolist()
encoding_columns = [t for t in train.columns if t not in prediction_columns]

In [318]:
# # encoding_columns
# new_id_test = [idd for idd in test.sku_name.unique().tolist() if idd not in train.sku_name.unique().tolist()]
# # encoding_columns
# new_id_train = [idd for idd in train.sku_name.unique().tolist() if idd not in test.sku_name.unique().tolist()]

In [319]:
sellin = ['sellin','sellin_channel_1','sellin_channel_2','sellin_channel_3','sellin_channel_4',
         'sellin_channel_5','sellin_channel_6','sellin_channel_7','sellin_channel_8']
sellout = ['sellout','sellout_channel_1','sellout_channel_2','sellout_channel_3','sellout_channel_4',
         'sellout_channel_5','sellout_channel_6','sellout_channel_7','sellout_channel_8','sellout_channel_9',
           'sellout_channel_10']
onhand= ['onhand_inventory','onhand_inventory_channel_1','onhand_inventory_channel_2','onhand_inventory_channel_3','onhand_inventory_channel_4',
         'onhand_inventory_channel_5','onhand_inventory_channel_6','onhand_inventory_channel_7','onhand_inventory_channel_8','onhand_inventory_channel_9',
           'onhand_inventory_channel_10']

In [320]:
# train.groupby('sku_name')[sellin[1:]].max().idxmax(axis=1) # other encoding
# train.groupby('sku_name')[sellin[1:]].min().idxmin(axis=1) # other encoding

# train.groupby(['sku_name','month'])[sellin[0]].sum().reset_index().pivot(index='sku_name', columns='month',values = 'sellin').idxmin(axis = 1).reset_index() # other encoding

In [321]:
df_train = train.copy()
df_test = test.copy()
df_train[sellin+sellout+onhand+['leftover_inventory','starting_inventory']] = df_train[sellin+sellout+onhand+['leftover_inventory','starting_inventory']]/1013

In [322]:
# df_train.columns

In [323]:
df_train['price0'] = df_train['price']//100
df_train['price1'] = (df_train['price']//10)%10
df_train['price2'] = df_train['price']%10
df_train['price3'] = df_train['price']//10

In [324]:
# Drop duplicated months
# Add last flag
## add weeks 

In [325]:
add_pivotal_sum = False
add_pivotal_max = False
encode_channels = False
add_pivotal_min = False
add_kmeans = False
# enode_channels = False
add_similar_month = False

In [326]:
if encode_channels:
        sku_name_channel_enc = df_train.groupby('sku_name')[sellin[1:]].max().idxmax(axis=1) # other encoding
        sku_name_channel_enc1 = df_train.groupby('sku_name')[sellin[1:]].min().idxmin(axis=1)# train.groupby('sku_name')[sellin[1:]].min().idxmin(axis=1) # other encoding
        df_train['sku_name_channel_enc'] = df_train['sku_name'].map(sku_name_channel_enc)
        df_train['sku_name_channel_enc1'] = df_train['sku_name'].map(sku_name_channel_enc1)
        df_test['sku_name_channel_enc'] = df_test['sku_name'].map(sku_name_channel_enc)
        df_test['sku_name_channel_enc1'] = df_test['sku_name'].map(sku_name_channel_enc1)
        df_test['sku_name_channel_enc1'].fillna('sellin_channel_1', inplace=True)
        df_test['sku_name_channel_enc'].fillna('sellin_channel_4', inplace=True)
        le = LE()
        df_train['sku_name_channel_enc1'] = le.fit_transform(df_train['sku_name_channel_enc1'])
        df_test['sku_name_channel_enc1'] = le.transform(df_test['sku_name_channel_enc1'])
        le = LE()
        df_train['sku_name_channel_enc'] = le.fit_transform(df_train['sku_name_channel_enc'])
        df_test['sku_name_channel_enc'] = le.transform(df_test['sku_name_channel_enc'])
        

In [327]:
if add_pivotal_sum:
        cols = ['month_'+str(i+1) for i in range(12) ]
        exp = df_train.groupby(['sku_name','month'])[sellin[0]].sum().reset_index().pivot(index='sku_name', columns='month',values = 'sellin').reset_index()
        exp.columns = ['sku_name']+ cols
        from sklearn.impute import KNNImputer
#         imputer = KNNImputer(n_neighbors=5)
#         exp[cols] = imputer.fit_transform(exp[cols])
        exp.fillna(0,inplace = True)
        exp.set_index('sku_name',inplace = True)
        for c in cols:
            df_test['sku_name_target_enc_sum_'+c] = df_test['sku_name'].map(exp[c])
            tmp = np.repeat(np.nan, df_train.shape[0])
            kf = KFold(n_splits = 5, shuffle=True,random_state = 1991032)
            for idx_1, idx_2 in kf.split(df_train):
                exp2 = df_train.iloc[idx_1].groupby(['sku_name','month'])[sellin[0]].sum().reset_index().pivot(index='sku_name', columns='month',values = 'sellin').reset_index()
                exp2.columns = ['sku_name']+ cols
                imputer = KNNImputer(n_neighbors=20)
                exp2[cols] = imputer.fit_transform(exp2[cols])
                exp2.set_index('sku_name',inplace = True)

                tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(exp2[c])
                df_train['sku_name_target_enc_sum_'+c] = tmp
        imputer = KNNImputer(n_neighbors=20)
        df_train[['sku_name_target_enc_sum_'+j for j in cols]] = imputer.fit_transform(df_train[['sku_name_target_enc_sum_'+j for j in cols]])
if add_pivotal_min:
        cols = ['month_'+str(i+1) for i in range(12) ]
        exp = df_train.groupby(['sku_name','month'])[sellin[0]].min().reset_index().pivot(index='sku_name', columns='month',values = 'sellin').reset_index()
        exp.columns = ['sku_name']+ cols
        from sklearn.impute import KNNImputer
#         imputer = KNNImputer(n_neighbors=5)
#         exp[cols] = imputer.fit_transform(exp[cols])
        exp.fillna(0,inplace = True)
    
        
        exp.set_index('sku_name',inplace = True)
        for c in cols:
            df_test['sku_name_target_enc_min_'+c] = df_test['sku_name'].map(exp[c])
            tmp = np.repeat(np.nan, df_train.shape[0])
            kf = KFold(n_splits = 5, shuffle=True,random_state = 1910325)
            for idx_1, idx_2 in kf.split(df_train):
                exp2 = df_train.iloc[idx_1].groupby(['sku_name','month'])[sellin[0]].min().reset_index().pivot(index='sku_name', columns='month',values = 'sellin').reset_index()
                exp2.columns = ['sku_name']+ cols
#                 imputer = KNNImputer(n_neighbors=5)
#                 exp2[cols] = imputer.fit_transform(exp2[cols])
                exp2.set_index('sku_name',inplace = True)

                tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(exp2[c])
                df_train['sku_name_target_enc_min_'+c] = tmp
        imputer = KNNImputer(n_neighbors=20)
        df_train[['sku_name_target_enc_min_'+j for j in cols]] = imputer.fit_transform(df_train[['sku_name_target_enc_min_'+j for j in cols]])
if add_pivotal_max:
        cols = ['month_'+str(i+1) for i in range(12) ]
        exp = df_train.groupby(['sku_name','month'])[sellin[0]].max().reset_index().pivot(index='sku_name', columns='month',values = 'sellin').reset_index()
        exp.columns = ['sku_name']+ cols
        from sklearn.impute import KNNImputer
#         imputer = KNNImputer(n_neighbors=5)
#         exp[cols] = imputer.fit_transform(exp[cols])
        exp.fillna(0,inplace = True)

        exp.set_index('sku_name',inplace = True)
        for c in cols:
            df_test['sku_name_target_enc_max_'+c] = df_test['sku_name'].map(exp[c])
            tmp = np.repeat(np.nan, df_train.shape[0])
            kf = KFold(n_splits = 5, shuffle=True,random_state = 9910325)
            for idx_1, idx_2 in kf.split(df_train):
                exp2 = df_train.iloc[idx_1].groupby(['sku_name','month'])[sellin[0]].max().reset_index().pivot(index='sku_name', columns='month',values = 'sellin').reset_index()
                exp2.columns = ['sku_name']+ cols
#                 imputer = KNNImputer(n_neighbors=5)
#                 exp2[cols] = imputer.fit_transform(exp2[cols])
                exp2.set_index('sku_name',inplace = True)

                tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(exp2[c])
                df_train['sku_name_target_enc_max_'+c] = tmp
        imputer = KNNImputer(n_neighbors=20)
        df_train[['sku_name_target_enc_max_'+j for j in cols]] = imputer.fit_transform(df_train[['sku_name_target_enc_max_'+j for j in cols]])


In [328]:
# cols = ['month_'+str(i+1) for i in range(12) ]
# exp = df_train.groupby(['sku_name','month'])[sellin[0]].sum().reset_index().pivot(index='sku_name', columns='month',values = 'sellin').reset_index()
# exp.columns = ['sku_name']+ cols
# from sklearn.impute import KNNImputer
# imputer = KNNImputer(n_neighbors=20)
# exp[cols] = imputer.fit_transform(exp[cols])
# train['onhand_inventory']

In [329]:
# sns.displot(
# exp[exp.isna().sum(axis = 1)==6]
#     .drop('sku_name',axis = 1).mean(axis = 1))
pivotal_sum_fea = ['sku_name_target_enc_sum_month_'+str(i+1) for i in range(12) ]
pivotal_min_fea = ['sku_name_target_enc_min_month_'+str(i+1) for i in range(12) ]
pivotal_max_fea = ['sku_name_target_enc_max_month_'+str(i+1) for i in range(12) ]
pivot_fea = pivotal_sum_fea + pivotal_max_fea + pivotal_min_fea
# new_fea = [f for f in X_train.columns if f not in pivot_fea ]
price_enc =  ['sku_namepriceenc6','sku_namepriceenc5','sku_namepriceenc4','sku_namepriceenc','sku_namepriceenc1','sku_namepriceenc2','sku_namepriceenc3']
sellin_enc =  ['sku_namesellinenc6','sku_namesellinenc5','sku_namesellinenc4','sku_namesellinenc','sku_namesellinenc1','sku_namesellinenc2','sku_namesellinenc3']
sellout_enc =  ['sku_nameselloutenc6','sku_nameselloutenc5','sku_nameselloutenc4','sku_nameselloutenc','sku_nameselloutenc1','sku_nameselloutenc2','sku_nameselloutenc3']
onhand_inventory_enc =  ['sku_nameonhand_inventoryenc6','sku_nameonhand_inventoryenc5','sku_nameonhand_inventoryenc4','sku_nameonhand_inventoryenc','sku_nameonhand_inventoryenc1','sku_nameonhand_inventoryenc2','sku_nameonhand_inventoryenc3']
starting_inventory_enc =  ['sku_namestarting_inventoryenc6','sku_namestarting_inventoryenc5','sku_namestarting_inventoryenc4','sku_namestarting_inventoryenc','sku_namestarting_inventoryenc1','sku_namestarting_inventoryenc2','sku_namestarting_inventoryenc3']
enc_features = price_enc + sellin_enc + sellout_enc + onhand_inventory_enc + starting_inventory_enc 

In [330]:
# new_fea = [f for f in X_train.columns if f not in pivot_fea+enc_features ]


In [331]:
# new_fea

In [332]:
# # # def quantile_encode(df1,df2):
# # # np.quantile(x,0.95)
# # # df_train.groupby(['sku_name','month'])[sellin[0]].apply(lambda x : np.quantile(x,0.95))

# # if add_similar_month:
# #         df_train['new_index'] = df_train['sku_name']+'_'+df_train['month'].astype(str)
#         df_test['new_index'] = df_test['sku_name']+'_'+df_test['month'].astype(str)
# #         new_df = df_train.groupby('new_index')['sellin'].mean()
# #         df_test['sku_name_target_enc_overfit'] = df_test['new_index'].map(new_df)
# #         df_test['sku_name_target_enc_overfit'].fillna((df_test['sku_name_target_enc_overfit'].mean()), inplace=True)
# #         tmp = np.repeat(np.nan, df_train.shape[0])
# #         kf = KFold(n_splits = 5, shuffle=True,random_state = 29910325)
# #         for idx_1, idx_2 in kf.split(df_train):
# #             target_mean = df_train.iloc[idx_1].groupby('new_index')['sellin'].mean()

# #             tmp[idx_2] = df_train['new_index'].iloc[idx_2].map(target_mean)
# #         df_train['sku_name_target_enc_overfit'] = tmp
# #         df_train['sku_name_target_enc_overfit'].fillna((df_train['sku_name_target_enc_overfit'].mean()), inplace=True)
# #         df_train.drop('new_index',axis = 1, inplace = True)
# #         df_test.drop('new_index',axis = 1, inplace = True)

In [333]:
def target_encode(df_train,df_test,target):
    sku_name_target_mean = df_train.groupby('sku_name')[target].mean()
    sku_name_target_max = df_train.groupby('sku_name')[target].max()
    sku_name_target_min = df_train.groupby('sku_name')[target].min()
    sku_name_target_std = df_train.groupby('sku_name')[target].std()
    sku_name_target_median = df_train.groupby('sku_name')[target].apply(lambda x : np.median(x))
    sku_name_target_q95 = df_train.groupby('sku_name')[target].apply(lambda x : np.quantile(x,0.95))
    sku_name_target_q05 = df_train.groupby('sku_name')[target].apply(lambda x : np.quantile(x,0.05))
    # sku_name_target_mean = df_train.groupby('sku_name')[target].mean()
    df_test['sku_name'+target+'enc'] = df_test['sku_name'].map(sku_name_target_mean)
    df_test['sku_name'+target+'enc'].fillna((df_test['sku_name'+target+'enc'].mean()), inplace=True)
    df_test['sku_name'+target+'enc1'] = df_test['sku_name'].map(sku_name_target_max)
    df_test['sku_name'+target+'enc1'].fillna((df_test['sku_name'+target+'enc1'].mean()), inplace=True)
    df_test['sku_name'+target+'enc2'] = df_test['sku_name'].map(sku_name_target_min)
    df_test['sku_name'+target+'enc2'].fillna((df_test['sku_name'+target+'enc2'].mean()), inplace=True)
    df_test['sku_name'+target+'enc3'] = df_test['sku_name'].map(sku_name_target_std)
    df_test['sku_name'+target+'enc3'].fillna((df_test['sku_name'+target+'enc3'].mean()), inplace=True)
    df_test['sku_name'+target+'enc4'] = df_test['sku_name'].map(sku_name_target_median)
    df_test['sku_name'+target+'enc4'].fillna((df_test['sku_name'+target+'enc4'].mean()), inplace=True)
    df_test['sku_name'+target+'enc5'] = df_test['sku_name'].map(sku_name_target_q95)
    df_test['sku_name'+target+'enc5'].fillna((df_test['sku_name'+target+'enc5'].mean()), inplace=True)
    df_test['sku_name'+target+'enc6'] = df_test['sku_name'].map(sku_name_target_q05)
    df_test['sku_name'+target+'enc6'].fillna((df_test['sku_name'+target+'enc6'].mean()), inplace=True)
    tmp =np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 19910325)
    for idx_1, idx_2 in kf.split(df_train):
        target_mean = df_train.iloc[idx_1].groupby('sku_name')[target].apply(lambda x : np.quantile(x,0.05))

        tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(target_mean)
    df_train['sku_name'+target+'enc6'] = tmp
    # from sklearn.model_selection import KFold
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 1991032)
    for idx_1, idx_2 in kf.split(df_train):
        target_mean = df_train.iloc[idx_1].groupby('sku_name')[target].apply(lambda x : np.quantile(x,0.95))

        tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(target_mean)
    df_train['sku_name'+target+'enc5'] = tmp



    # from sklearn.model_selection import KFold
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 1991035)
    for idx_1, idx_2 in kf.split(df_train):
        target_mean = df_train.iloc[idx_1].groupby('sku_name')[target].apply(lambda x : np.median(x))

        tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(target_mean)
    df_train['sku_name'+target+'enc4'] = tmp



    # from sklearn.model_selection import KFold
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 1910325)
    for idx_1, idx_2 in kf.split(df_train):
        target_mean = df_train.iloc[idx_1].groupby('sku_name')[target].mean()

        tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(target_mean)
    df_train['sku_name'+target+'enc'] = tmp



    # from sklearn.model_selection import KFold
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 1991025)
    for idx_1, idx_2 in kf.split(df_train):
        target_max = df_train.iloc[idx_1].groupby('sku_name')[target].max()

        tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(target_max)
    df_train['sku_name'+target+'enc1'] = tmp
    # from sklearn.model_selection import KFold/
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 1990325)
    for idx_1, idx_2 in kf.split(df_train):
        target_min = df_train.iloc[idx_1].groupby('sku_name')[target].min()

        tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(target_min)
    df_train['sku_name'+target+'enc2'] = tmp
    # from sklearn.model_selection import KFold
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 19910325)
    for idx_1, idx_2 in kf.split(df_train):
        target_std = df_train.iloc[idx_1].groupby('sku_name')[target].std()

        tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(target_std)
    df_train['sku_name'+target+'enc3'] = tmp
    df_train['sku_name'+target+'enc'].fillna((df_train['sku_name'+target+'enc'].mean()), inplace=True)
    df_train['sku_name'+target+'enc1'].fillna((df_train['sku_name'+target+'enc1'].mean()), inplace=True)
    df_train['sku_name'+target+'enc2'].fillna((df_train['sku_name'+target+'enc2'].mean()), inplace=True)
    df_train['sku_name'+target+'enc3'].fillna((df_train['sku_name'+target+'enc3'].mean()), inplace=True)
    df_train['sku_name'+target+'enc4'].fillna((df_train['sku_name'+target+'enc4'].mean()), inplace=True)
    df_train['sku_name'+target+'enc5'].fillna((df_train['sku_name'+target+'enc5'].mean()), inplace=True)
    df_train['sku_name'+target+'enc6'].fillna((df_train['sku_name'+target+'enc6'].mean()), inplace=True)
    return df_train, df_test
# train.groupby(['sku_name','month'])[sellin[0]].sum().reset_index().pivot(index='sku_name', columns='month',values = 'sellin').idxmax(axis = 1) # other encoding
# train.groupby(['sku_name','month'])[sellin[0]].sum().reset_index().pivot(index='sku_name', columns='month',values = 'sellin').idxmin(axis = 1).reset_index() # other encoding
    

In [334]:
def target_encode_p(df_train,df_test,target):
    sku_name_target_mean = df_train.groupby('sku_name')[target].mean().round().astype('int16')
    sku_name_target_max = df_train.groupby('sku_name')[target].max().round().astype('int16')
    sku_name_target_min = df_train.groupby('sku_name')[target].min().round().astype('int16')
    sku_name_target_std = df_train.groupby('sku_name')[target].std()
    sku_name_target_median = df_train.groupby('sku_name')[target].apply(lambda x : np.median(x).round().astype('int16'))
    sku_name_target_q95 = df_train.groupby('sku_name')[target].apply(lambda x : np.quantile(x,0.95).round().astype('int16'))
    sku_name_target_q05 = df_train.groupby('sku_name')[target].apply(lambda x : np.quantile(x,0.05).round().astype('int16'))
    # sku_name_target_mean = df_train.groupby('sku_name')[target].mean()
    df_test['sku_name'+target+'enc'] = df_test['sku_name'].map(sku_name_target_mean)
    df_test['sku_name'+target+'enc'].fillna((df_test['sku_name'+target+'enc'].mean().round().astype('int16')), inplace=True)
    df_test['sku_name'+target+'enc1'] = df_test['sku_name'].map(sku_name_target_max)
    df_test['sku_name'+target+'enc1'].fillna((df_test['sku_name'+target+'enc1'].mean().round().astype('int16')), inplace=True)
    df_test['sku_name'+target+'enc2'] = df_test['sku_name'].map(sku_name_target_min)
    df_test['sku_name'+target+'enc2'].fillna((df_test['sku_name'+target+'enc2'].mean().round().astype('int16')), inplace=True)
    df_test['sku_name'+target+'enc3'] = df_test['sku_name'].map(sku_name_target_std)
    df_test['sku_name'+target+'enc3'].fillna((df_test['sku_name'+target+'enc3']), inplace=True)
    df_test['sku_name'+target+'enc4'] = df_test['sku_name'].map(sku_name_target_median)
    df_test['sku_name'+target+'enc4'].fillna((df_test['sku_name'+target+'enc4'].mean().round().astype('int16')), inplace=True)
    df_test['sku_name'+target+'enc5'] = df_test['sku_name'].map(sku_name_target_q95)
    df_test['sku_name'+target+'enc5'].fillna((df_test['sku_name'+target+'enc5'].mean().round().astype('int16')), inplace=True)
    df_test['sku_name'+target+'enc6'] = df_test['sku_name'].map(sku_name_target_q05)
    df_test['sku_name'+target+'enc6'].fillna((df_test['sku_name'+target+'enc6'].mean().round().astype('int16')), inplace=True)
    tmp =np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 19910325)
    for idx_1, idx_2 in kf.split(df_train):
        target_mean = df_train.iloc[idx_1].groupby('sku_name')[target].apply(lambda x : np.quantile(x,0.05).round().astype('int16'))

        tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(target_mean)
    df_train['sku_name'+target+'enc6'] = tmp
    # from sklearn.model_selection import KFold
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 1991032)
    for idx_1, idx_2 in kf.split(df_train):
        target_mean = df_train.iloc[idx_1].groupby('sku_name')[target].apply(lambda x : np.quantile(x,0.95).round().astype('int16'))

        tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(target_mean)
    df_train['sku_name'+target+'enc5'] = tmp



    # from sklearn.model_selection import KFold
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 1991035)
    for idx_1, idx_2 in kf.split(df_train):
        target_mean = df_train.iloc[idx_1].groupby('sku_name')[target].apply(lambda x : np.median(x).round().astype('int16'))

        tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(target_mean)
    df_train['sku_name'+target+'enc4'] = tmp



    # from sklearn.model_selection import KFold
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 1910325)
    for idx_1, idx_2 in kf.split(df_train):
        target_mean = df_train.iloc[idx_1].groupby('sku_name')[target].mean().round().astype('int16')

        tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(target_mean)
    df_train['sku_name'+target+'enc'] = tmp



    # from sklearn.model_selection import KFold
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 1991025)
    for idx_1, idx_2 in kf.split(df_train):
        target_max = df_train.iloc[idx_1].groupby('sku_name')[target].max().round().astype('int16')

        tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(target_max)
    df_train['sku_name'+target+'enc1'] = tmp
    # from sklearn.model_selection import KFold/
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 1990325)
    for idx_1, idx_2 in kf.split(df_train):
        target_min = df_train.iloc[idx_1].groupby('sku_name')[target].min().round().astype('int16')

        tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(target_min)
    df_train['sku_name'+target+'enc2'] = tmp
    # from sklearn.model_selection import KFold
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 5, shuffle=True,random_state = 19910325)
    for idx_1, idx_2 in kf.split(df_train):
        target_std = df_train.iloc[idx_1].groupby('sku_name')[target].std()

        tmp[idx_2] = df_train['sku_name'].iloc[idx_2].map(target_std)
    df_train['sku_name'+target+'enc3'] = tmp
    df_train['sku_name'+target+'enc'].fillna((df_train['sku_name'+target+'enc'].mean().round().astype('int16')), inplace=True)
    df_train['sku_name'+target+'enc1'].fillna((df_train['sku_name'+target+'enc1'].mean().round().astype('int16')), inplace=True)
    df_train['sku_name'+target+'enc2'].fillna((df_train['sku_name'+target+'enc2'].mean().round().astype('int16')), inplace=True)
    df_train['sku_name'+target+'enc3'].fillna((df_train['sku_name'+target+'enc3'].mean()), inplace=True)
    df_train['sku_name'+target+'enc4'].fillna((df_train['sku_name'+target+'enc4'].mean().round().astype('int16').round().astype('int16')), inplace=True)
    df_train['sku_name'+target+'enc5'].fillna((df_train['sku_name'+target+'enc5'].mean().round().astype('int16')), inplace=True)
    df_train['sku_name'+target+'enc6'].fillna((df_train['sku_name'+target+'enc6'].mean().round().astype('int16')), inplace=True)
    return df_train, df_test
# train.groupby(['sku_name','month'])[sellin[0]].sum().reset_index().pivot(index='sku_name', columns='month',values = 'sellin').idxmax(axis = 1) # other encoding
# train.groupby(['sku_name','month'])[sellin[0]].sum().reset_index().pivot(index='sku_name', columns='month',values = 'sellin').idxmin(axis = 1).reset_index() # other encoding
    

In [335]:
df_train,df_test = target_encode(df_train,df_test,'sellin')
df_train,df_test = target_encode(df_train,df_test,'sellout')
# df_train,df_test = target_encode_p(df_train,df_test,'price0')
# df_train,df_test = target_encode_p(df_train,df_test,'price1')
# df_train,df_test = target_encode_p(df_train,df_test,'price2')
# df_train,df_test = target_encode_p(df_train,df_test,'price3')
df_train,df_test = target_encode(df_train,df_test,'price')

# df_train,df_test = target_encode(df_train,df_test,'starting_inventory')
# df_train,df_test = target_encode(df_train,df_test,'leftover_inventory')
pcl = df_train.groupby(['sku_name','year','month'])['product_lifecycle_stage'].apply(lambda x : x.unique()[0]).reset_index()
pcl_test = df_train.groupby('sku_name')['product_lifecycle_stage'].apply(lambda x:x.values[-1])
df_train.drop('product_lifecycle_stage',axis = 1,inplace = True)
new_target = df_train.groupby(['sku_name','year','month'])['sellin'].sum().reset_index()
df_train = df_train.merge(pcl,on = ['sku_name','year','month'],how = 'left')
df_test['product_lifecycle_stage'] = df_test['sku_name'].map(pcl_test)
target = 'sellin'
prediction_columns = df_test.columns.tolist()
encoding_columns = [t for t in df_train.columns if t not in prediction_columns]
# X = df_train.drop(encoding_columns+[target],axis=1)
# y = df_train[target]
df_train = df_train.drop(encoding_columns+[target],axis=1).merge(new_target,on = ['sku_name','year','month'],how = 'left').drop_duplicates()

In [336]:
all_data = pd.concat([df_train[prediction_columns],df_test[prediction_columns]])

In [337]:

le = LE()
all_data['enc_sku'] = le.fit_transform(all_data['sku_name'])
df_train['enc_sku'] = all_data.iloc[:df_train.shape[0]]['enc_sku']
df_test['enc_sku'] = all_data.iloc[df_train.shape[0]:]['enc_sku']

le = LE()
all_data['product_lifecycle_stage'] = le.fit_transform(all_data['product_lifecycle_stage'])
df_train['product_lifecycle_stage'] = all_data.iloc[:df_train.shape[0]]['product_lifecycle_stage']
df_test['product_lifecycle_stage'] = all_data.iloc[df_train.shape[0]:]['product_lifecycle_stage']
# df_test['enc_sku'] = le.transform(df_test['sku_name'])

In [338]:
# df_train.info()

In [339]:
add_similar_products = True
if add_similar_month:
        df_train['new_index'] = df_train['sku_name']+'_'+df_train['month'].astype(str)
        df_test['new_index'] = df_test['sku_name']+'_'+df_test['month'].astype(str)
        new_df = df_train.groupby('new_index')['sellin'].mean()
        df_test['sku_name_target_enc_overfit'] = df_test['new_index'].map(new_df)
        df_test['sku_name_target_enc_overfit'].fillna((df_test['sku_name_target_enc_overfit'].mean()), inplace=True)
        tmp = np.repeat(np.nan, df_train.shape[0])
        kf = KFold(n_splits = 5, shuffle=True,random_state = 29910325)
        for idx_1, idx_2 in kf.split(df_train):
            target_mean = df_train.iloc[idx_1].groupby('new_index')['sellin'].mean()

            tmp[idx_2] = df_train['new_index'].iloc[idx_2].map(target_mean)
        df_train['sku_name_target_enc_overfit'] = tmp
        df_train['sku_name_target_enc_overfit'].fillna((df_train['sku_name_target_enc_overfit'].mean()), inplace=True)
        df_train.drop('new_index',axis = 1, inplace = True)
        df_test.drop('new_index',axis = 1, inplace = True)
# What i suppose is similar products



In [340]:
price_cols = ['sku_nameprice0enc',
 'sku_nameprice0enc1',
 'sku_nameprice0enc2',
 'sku_nameprice0enc3',
 'sku_nameprice0enc4',
 'sku_nameprice0enc5',
 'sku_nameprice0enc6',
 'sku_nameprice1enc',
 'sku_nameprice1enc1',
 'sku_nameprice1enc2',
 'sku_nameprice1enc3',
 'sku_nameprice1enc4',
 'sku_nameprice1enc5',
 'sku_nameprice1enc6',
 'sku_nameprice2enc',
 'sku_nameprice2enc1',
 'sku_nameprice2enc2',
 'sku_nameprice2enc3',
 'sku_nameprice2enc4',
 'sku_nameprice2enc5',
 'sku_nameprice2enc6',
 'sku_nameprice3enc',
 'sku_nameprice3enc1',
 'sku_nameprice3enc2',
 'sku_nameprice3enc3',
 'sku_nameprice3enc4',
 'sku_nameprice3enc5',
 'sku_nameprice3enc6']

In [233]:
# sc = (df_train.select_dtypes(exclude = 'object')%1 == 0).all()

# df_train.select_dtypes(include = 'int')

In [341]:
# df_train.loc[:,sc[sc==1].index] = df_train[sc[sc==1].index].astype('int')
# df_test.loc[:,df_test.columns.isin(sc[sc==1].index)] = df_test.loc[:,df_test.columns.isin(sc[sc==1].index)].astype('int')

In [342]:
scale = df_test.select_dtypes(include = 'float64').columns.tolist()

In [343]:
# df_train

In [344]:
from sklearn.preprocessing import RobustScaler
rs = RobustScaler()
for s in scale:
    df_train[s] = rs.fit_transform(df_train[s].values.reshape(-1, 1))
    df_test[s] = rs.transform(df_test[s].values.reshape(-1, 1))

In [345]:
# # df_train['sellin'].min()
# # df_test.shape
# # df_train
# target = 'sellin'
# prediction_columns = df_test.columns.tolist()
# encoding_columns = [t for t in df_train.columns if t not in prediction_columns]
# X = df_train.drop(encoding_columns+[target,'sku_name'],axis=1)
# y = df_train[target]
# # max_ = X['sku_name_target_enc1']

In [346]:
# target = 'sellin'
# prediction_columns = df_test.columns.tolist()
# encoding_columns = [t for t in df_train.columns if t not in prediction_columns]
# X = df_train.drop(encoding_columns+[target],axis=1)
# y = df_train[target]
# # max_ = X['sku_name_target_enc1']


In [347]:
# cats

In [348]:
def rmspe(y_true, y_pred):
    return  np.mean(np.abs(y_true - y_pred))*1013#(np.sqrt(np.mean(np.square((y_true - y_pred) / (y_true+2)))))

def feval_RMSPE(preds, lgbm_train):
    labels = lgbm_train.get_label()
    return 'RMSPE', round(rmspe(y_true = labels, y_pred = preds),0), False

params = {
      "objective": "mae", 
      "metric": "mae", 
      "boosting_type": "gbdt",
      'early_stopping_rounds': 30,
      'learning_rate': 0.01,
#       'lambda_l1': 1,
#       'lambda_l2': 1,
#       'feature_fraction': 0.8,
#       'bagging_fraction': 0.8,
  }

In [349]:
kfolds = 10
from sklearn.model_selection import KFold
kf = KFold(n_splits=kfolds, random_state=19901028, shuffle=True)
oof = pd.DataFrame()                 # out-of-fold result
models = []                          # models
scores = 0.0   

features_importance= pd.DataFrame({'Feature':[], 'Importance':[]})
# validation score

In [350]:
target = 'sellin'
prediction_columns = df_test.columns.tolist()
encoding_columns = [t for t in df_train.columns if t not in prediction_columns]
X = df_train.drop(encoding_columns+[target,'sku_name'],axis=1)
y = df_train[target]
# max_ = X['sku_name_target_enc1']

In [351]:
cats =[]
# for c in X.columns:
#     if X[c].dtype == 'int':
#         cats.append(X.columns.get_loc(c))

In [352]:
%%time
for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):

    print("Fold :", fold+1)
    
    # create dataset
    X_train, y_train = X.loc[trn_idx], y[trn_idx]
    X_valid, y_valid = X.loc[val_idx], y[val_idx]
    
    #RMSPE weight
#     weights = 1/(w_train)
    lgbm_train = lgbm.Dataset(X_train,y_train)#,weight = weights)

#     weights = 1/(w_valid)
    lgbm_valid = lgbm.Dataset(X_valid,y_valid,reference = lgbm_train)#,weight = weights)
    
    # model 
    model = lgbm.train(params=params,
                      train_set=lgbm_train,
                      valid_sets=[lgbm_train, lgbm_valid],
                      num_boost_round=15000,         
#                       feval='mae',
                      verbose_eval=100,
                      categorical_feature = cats                
                     )
    
    # validation 
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    features = X_train.columns
    RMSPE = round(rmspe(y_true = y_valid, y_pred = y_pred),3)
    print(f'Performance of the　prediction: , RMSPE: {RMSPE}')
    fold_importance_df= pd.DataFrame({'Feature':[], 'Importance':[]})
    fold_importance_df['Feature']= features
    fold_importance_df['Importance']= model.feature_importance()
    fold_importance_df["fold"] = fold + 1
    features_importance = pd.concat([features_importance, fold_importance_df], axis=0)
    #keep scores and models
    scores += RMSPE / kfolds
    models.append(model)
    print("*" * 100)

In [353]:
display_importances(features_importance)


In [355]:
X_test = df_test[X_train.columns.tolist()].copy()
# X_train
# len(models)

In [356]:
df_test['Item_ID'] = df_test['sku_name'] +'_'+ df_test['month'].astype(str)+ '_'+df_test['year'].astype(str)
y_pred = df_test[['Item_ID']]
target = np.zeros(len(X_test))
target2 = np.zeros(len(X_test))
#light gbm models
for model in models:
    pred = model.predict(X_test[X_valid.columns], num_iteration=model.best_iteration)
    target += pred / len(models)
# X_test['cum_disc'] = 0
# for model in models:
#     pred = model.predict(X_test[X_valid.columns], num_iteration=model.best_iteration)
#     target2 += pred / len(models)
# target = (target+target2)/2
y_pred = y_pred.assign(target_y= target)
sub = ss.merge(y_pred,on = 'Item_ID')
sub['target'] = sub['target_y']*1013
sub['target'] = (sub['target'].astype('int') )
sub = sub[['Item_ID','target']]

In [291]:
# models[5].predict(X_test[X_valid.columns], num_iteration=model.best_iteration)*1013
# sns.distplot(df_train.sellin)
# sns.distplot(sub.target)

In [357]:
sub

In [358]:
sub.to_csv('submission_41_neg.csv',index = False)

In [ ]:
# len(models)
ss['sku_month'] = ss['Item_ID'].apply(lambda x: x.split('_')[0])+'_'+ss['Item_ID'].apply(lambda x: x.split('_')[1])

In [ ]:
df_train['sku_month'] = df_train['sku_name']+'_'+df_train['month'].astype(str)

In [ ]:
# ss.loc[ss.sku_month.isin(df_train.sku_month)]
plt.plot(np.log(df_train['sellin'].sort_values().values[3:-301]))

In [ ]:
ss.shape[0]

In [ ]:
# df_test.apply(lambda x: '_'.join(x['sku_name'],x['month']))


# Clusters

In [ ]:

cluster_range = range(1,15)
cluster_errors = []
for num_clusters in cluster_range:
    clusters = KMeans(num_clusters, n_init = 5)
    clusters.fit(df_train[sellin+sellout+onhand+['month','year']])
    labels = clusters.labels_
    centroids = clusters.cluster_centers_
    cluster_errors.append(clusters.inertia_)

clusters_df = pd.DataFrame({"num_clusters": cluster_range, "cluster_errors": cluster_errors})
# clusters_df
from matplotlib import cm

plt.figure(figsize=(12,6))
plt.plot( clusters_df.num_clusters, clusters_df.cluster_errors, marker = "o" )

In [ ]:
# np.unique(labels)

In [133]:
!pip install flaml

In [134]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)
from flaml import AutoML
automl = AutoML()

In [ ]:
automl.fit(X_train, y_train, task="regression",metric='mae',time_budget=898)

In [155]:
print('Best ML learner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best mae on validation data: {0:.4g}'.format(automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

In [158]:
df_test['Item_ID'] = df_test['sku_name'] +'_'+ df_test['month'].astype(str)+ '_'+df_test['year'].astype(str)
y_pred = df_test[['Item_ID']]
target = np.zeros(len(X_test))

#light gbm models

pred = automl.predict(df_test[X_train.columns])
target = pred 
y_pred = y_pred.assign(target_y= target)
sub = ss.merge(y_pred,on = 'Item_ID')
sub['target'] = sub['target_y']*1013
sub['target'] = sub['target'].astype('int') 
sub = sub[['Item_ID','target']]

In [160]:
sub.to_csv('submission_21.csv',index = False)